In [17]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from df2gspread import df2gspread as d2g


In [18]:
my_email = 'zhevakin6389@gmail.com'

path_to_credential = '/content/tets-task-cruise-11157e60f336.json'
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)
my_name = gspread.authorize(credentials)

url = 'https://docs.google.com/spreadsheets/d/1ideR5UwlV5hZt3PL5gzTM_99hic_1xJd5e4A801BEuc/edit#gid=0'

sheet = my_name.open_by_url(url)

sheet_1 = sheet.get_worksheet(0)

sheet_2 = sheet.get_worksheet(1)

sheet_3 = sheet.get_worksheet(2)



In [19]:
employee_plans = sheet_1.get_all_records()
cv_plan = sheet_2.get_all_records()
work_days = sheet_3.get_all_records()

employee_plans_df = pd.DataFrame(employee_plans)
employee_plans_df['month'] = pd.to_datetime(employee_plans_df['month'])
employee_plans_df['month'] = employee_plans_df['month'].fillna(method = 'ffill')
employee_plans_df['for_merge'] = employee_plans_df['month'].dt.month

employee_plans_df['revenue_plan'] = employee_plans_df['revenue_plan'].astype(str)
employee_plans_df['revenue_plan'] = employee_plans_df['revenue_plan'].apply(lambda x: x.replace(' ', ''))
employee_plans_df['revenue_plan'] = employee_plans_df['revenue_plan'].astype(float)

work_days_df = pd.DataFrame(work_days)
work_days_df = work_days_df.rename(columns = lambda x: x.strip())
work_days_df['working_month'] = pd.to_datetime(work_days_df['working_month'])
work_days_df['for_merge'] = work_days_df['working_month'].dt.month

cv_plan_df = pd.DataFrame(cv_plan)
cv_plan_df['date_from'] = pd.to_datetime(cv_plan_df['date_from'])
cv_plan_df['date_to'] = cv_plan_df['date_from'] + pd.offsets.MonthEnd(2)
cv_plan_df['for_merge'] = cv_plan_df.apply(lambda row: [row['date_from'].month, row['date_to'].month] , axis = 1)
cv_plan_df = cv_plan_df.explode('for_merge')

In [20]:
cv_plan_df

,date_from,date_to,cv_plan,for_merge
0,2022-01-01,2022-02-28,0.6,1
0,2022-01-01,2022-02-28,0.6,2
1,2022-03-01,2022-04-30,0.7,3
1,2022-03-01,2022-04-30,0.7,4


In [21]:
general_table = employee_plans_df.merge(cv_plan_df, on = 'for_merge', how = 'left').merge(work_days_df, on = 'for_merge', how = 'left')
general_table['sales_plan'] =  general_table['revenue_plan'] / general_table['avr_check_plan']
general_table['leads_plan'] =  general_table['sales_plan'] / general_table['cv_plan']
general_table['leads_per_day_plan'] =  general_table['leads_plan'] / general_table['working_days']
general_table['revenue_per_day_plan'] =  general_table['revenue_plan'] / general_table['working_days']
general_table['revenue_per_day_plan'] =  general_table['revenue_plan'] / general_table['working_days']
general_table['for_merge'] = general_table['for_merge'].astype(int)
general_table = round(general_table[['month', 'id', 'revenue_plan', 'avr_check_plan','cv_plan', 'sales_plan', 'leads_plan', 'leads_per_day_plan', 'revenue_per_day_plan']],2)
general_table['month'] = general_table['month'].astype(str)


In [22]:
general_table

,month,id,revenue_plan,avr_check_plan,cv_plan,sales_plan,leads_plan,leads_per_day_plan,revenue_per_day_plan
0,2022-02-01,9905,617025.0,10000,0.6,61.70,102.84,5.71,34279.17
1,2022-02-01,14719,617025.0,10000,0.6,61.70,102.84,5.71,34279.17
2,2022-02-01,31765,493620.0,10000,0.6,49.36,82.27,4.57,27423.33
3,2022-02-01,36617,617025.0,10000,0.6,61.70,102.84,5.71,34279.17
4,2022-02-01,30521,617025.0,10000,0.6,61.70,102.84,5.71,34279.17
...,...,...,...,...,...,...,...,...,...
100,2022-04-01,60606,514112.0,14000,0.7,36.72,52.46,2.38,23368.73
101,2022-04-01,61066,464112.0,14000,0.7,33.15,47.36,2.15,21096.00
102,2022-04-01,59326,514112.0,14000,0.7,36.72,52.46,2.38,23368.73
103,2022-04-01,61484,349940.0,14000,0.7,25.00,35.71,1.62,15906.36


In [23]:
spreadsheet = my_name.create('New_Zhev')
spreadsheet_url = spreadsheet.url
worksheet = spreadsheet.sheet1
values = [general_table.columns.tolist()] + general_table.values.tolist()
worksheet.update('A1', values)
spreadsheet.share(spreadsheet_url, perm_type='anyone', role='writer')


<ipython-input-23-cccf58124600>:5: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', values)


<Response [200]>

In [24]:
spreadsheet_url

'https://docs.google.com/spreadsheets/d/14GQSEE9zJ_5SIiwQ0w0j10W6vudmKBmUNLuwZlfZ4dM'